# Import library.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

# Downlaod Data

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


trainset = torchvision.datasets.CIFAR10( root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader( trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Model definition.

In [3]:
cfg = {'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 
                 'M', 512, 512, 512, 'M']}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)



In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG('VGG16').to(device)
MODELNAME = 'CIFAR10-VGG16.model'
best_acc = 0
start_epoch = 0 
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Create train and test function.

In [6]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print( 'Train', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (train_loss,
                                        100.*correct/total, correct, total))


In [7]:
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print('Test:', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss, 
                                  100.*correct/total, correct, total))

    
    acc = 100.*correct/total
    if acc > best_acc:
        torch.save(model.state_dict(), MODELNAME)
        best_acc = acc
        print('--> Saved model')

# Train process and test model.

In [8]:
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Loss: 765.661 | Acc: 44.626% (22313/50000)
Test: Loss: 132.043 | Acc: 54.760% (5476/10000)
--> Saved model

Epoch: 1
Train Loss: 484.692 | Acc: 65.744% (32872/50000)
Test: Loss: 84.563 | Acc: 69.580% (6958/10000)
--> Saved model

Epoch: 2
Train Loss: 382.741 | Acc: 73.488% (36744/50000)
Test: Loss: 74.482 | Acc: 74.470% (7447/10000)
--> Saved model

Epoch: 3
Train Loss: 324.880 | Acc: 77.742% (38871/50000)
Test: Loss: 72.812 | Acc: 75.910% (7591/10000)
--> Saved model

Epoch: 4
Train Loss: 287.714 | Acc: 80.254% (40127/50000)
Test: Loss: 55.146 | Acc: 81.250% (8125/10000)
--> Saved model

Epoch: 5
Train Loss: 256.830 | Acc: 82.430% (41215/50000)
Test: Loss: 58.205 | Acc: 80.270% (8027/10000)

Epoch: 6
Train Loss: 235.677 | Acc: 83.896% (41948/50000)
Test: Loss: 53.321 | Acc: 81.670% (8167/10000)
--> Saved model

Epoch: 7
Train Loss: 214.828 | Acc: 85.414% (42707/50000)
Test: Loss: 48.606 | Acc: 83.290% (8329/10000)
--> Saved model

Epoch: 8
Train Loss: 199.117 | Acc: 86.432% (432